#### Steps in this notebook
- Clean Cricsheet data, create basic model
- Run basic model for testing

In [8]:
import pandas as pd
import numpy as np
import math

import sklearn

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

In [9]:
cricsheet_df=pd.read_csv('./data/all_matches.csv')

/Users/kishan/Desktop/IPL/.ipl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
###Load input format dataframe

# test_df=pd.read_csv('./submission/inputFile.csv')

# test_df

##Venue needs to go through standardizer
## innigns is either 1 or 2
## Batting team and bowling team better to go through standardizer, since KXIP has changed names between seasons
## Batsmen list must be minimum 2 and bowlers list must be minimum 2
## Need to standardize batter and bowlers
## Batting and bowling types of batters and bowlers to be inferred from custom module

#### Subsetting the given data

- Remove seasons before 2018
- Remove super over innings (innings>=3)
- Remove unnecessary columns

In [10]:
##no use of other wicket type and other player dismissed columns
assert cricsheet_df.loc[~pd.isnull(cricsheet_df['other_wicket_type'])].shape[0]==0
assert cricsheet_df.loc[~pd.isnull(cricsheet_df['other_player_dismissed'])].shape[0]==0

In [11]:
cricsheet_df.drop(columns=['other_wicket_type','other_player_dismissed'],axis=1,inplace=True)

In [12]:
cricsheet_df['start_date']=pd.to_datetime(cricsheet_df['start_date'])

In [13]:
##Filter from 2018 April 7th
cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all')

<ipython-input-13-ff7ecb1db089>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  cricsheet_df.loc[cricsheet_df['start_date']>='2018','start_date'].describe(include='all')


count                   48514
unique                    165
top       2020-10-18 00:00:00
freq                      520
first     2018-04-07 00:00:00
last      2021-04-27 00:00:00
Name: start_date, dtype: object

In [14]:
cdf=cricsheet_df.loc[cricsheet_df['start_date']>='2018'].reset_index(drop=True)

### EDA

In [15]:
##List of all venues in this
set(cdf['venue'])

{'Arun Jaitley Stadium',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'Dubai International Cricket Stadium',
 'Eden Gardens',
 'Holkar Cricket Stadium',
 'M.Chinnaswamy Stadium',
 'MA Chidambaram Stadium',
 'MA Chidambaram Stadium, Chepauk, Chennai',
 'Maharashtra Cricket Association Stadium',
 'Narendra Modi Stadium, Ahmedabad',
 'Punjab Cricket Association IS Bindra Stadium',
 'Rajiv Gandhi International Stadium',
 'Sawai Mansingh Stadium',
 'Sharjah Cricket Stadium',
 'Sheikh Zayed Stadium',
 'Wankhede Stadium',
 'Wankhede Stadium, Mumbai'}

#### There are 231 different IPL players across teams from 2018. 

In [16]:
##List of all unique players in the dataset
ipl_players=cdf.loc[:,['striker','non_striker','bowler']].drop_duplicates().melt()['value'].drop_duplicates().tolist()

len(ipl_players),ipl_players

(243,
 ['V Kohli',
  'PA Patel',
  'MM Ali',
  'AB de Villiers',
  'SO Hetmyer',
  'S Dube',
  'C de Grandhomme',
  'NA Saini',
  'YS Chahal',
  'UT Yadav',
  'Mohammed Siraj',
  'SR Watson',
  'AT Rayudu',
  'SK Raina',
  'KM Jadhav',
  'RA Jadeja',
  'DA Warner',
  'JM Bairstow',
  'V Shankar',
  'YK Pathan',
  'MK Pandey',
  'CA Lynn',
  'N Rana',
  'RV Uthappa',
  'KD Karthik',
  'AD Russell',
  'Shubman Gill',
  'PP Shaw',
  'S Dhawan',
  'SS Iyer',
  'CA Ingram',
  'RR Pant',
  'KMA Paul',
  'AR Patel',
  'R Tewatia',
  'RG Sharma',
  'Q de Kock',
  'SA Yadav',
  'KA Pollard',
  'Yuvraj Singh',
  'HH Pandya',
  'KH Pandya',
  'BCJ Cutting',
  'MJ McClenaghan',
  'Rasikh Salam',
  'KL Rahul',
  'MA Agarwal',
  'CH Gayle',
  'SN Khan',
  'N Pooran',
  'Mandeep Singh',
  'AM Rahane',
  'JC Buttler',
  'SV Samson',
  'SPD Smith',
  'RA Tripathi',
  'BA Stokes',
  'K Gowtham',
  'JC Archer',
  'JD Unadkat',
  'S Gopal',
  'DS Kulkarni',
  'MS Dhoni',
  'DJ Bravo',
  'SP Narine',
  'DA

In [164]:
# pd.DataFrame(ipl_players,columns=['players_2021']).to_excel('./data/players_2021.xlsx',index=False)

In [17]:
### Format seasons
print(set(cdf['season']))

cdf['season']=cdf['season'].astype('str')
print(set(cdf['season']))

{2018, 2019, '2021', 2021, '2019', '2018', '2020/21'}
{'2019', '2020/21', '2018', '2021'}


In [18]:
##Remove IPL 2020

#cdf=cdf.loc[cdf['season']!='2020/21']

In [19]:
##Remove super overs and repeated super over instance from MI KXIP
cdf=cdf.loc[cdf['innings']<=2]

In [20]:
##Which venues need tweaking? For purposes of training
cdf['venue'].value_counts(ascending=False)

Dubai International Cricket Stadium                    6338
Sheikh Zayed Stadium                                   5294
Wankhede Stadium                                       3902
Eden Gardens                                           3802
Rajiv Gandhi International Stadium                     3609
Arun Jaitley Stadium                                   3344
Sawai Mansingh Stadium                                 3286
M.Chinnaswamy Stadium                                  3246
Sharjah Cricket Stadium                                2878
MA Chidambaram Stadium, Chepauk, Chennai               2449
Punjab Cricket Association IS Bindra Stadium           2429
Wankhede Stadium, Mumbai                               2427
MA Chidambaram Stadium                                 2130
Maharashtra Cricket Association Stadium                1467
Holkar Cricket Stadium                                  873
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium     491
Narendra Modi Stadium, Ahmedabad        

In [25]:
cdf.loc[cdf['venue']=='MA Chidambaram Stadium, Chepauk, Chennai','venue']='MA Chidambaram Stadium'

cdf.loc[cdf['venue']=='Narendra Modi Stadium, Ahmedabad','venue']='Narendra Modi Stadium'
cdf.loc[cdf['venue']=='Wankhede Stadium, Mumbai','venue']='Wankhede Stadium'

#### Change jaipur to Ahmedabad

In [26]:
cdf.loc[cdf['venue']=='Sawai Mansingh Stadium','venue']='Narendra Modi Stadium'

In [27]:
cdf['venue'].value_counts(ascending=False)

Dubai International Cricket Stadium                    6338
Wankhede Stadium                                       6329
Sheikh Zayed Stadium                                   5294
MA Chidambaram Stadium                                 4579
Eden Gardens                                           3802
Narendra Modi Stadium                                  3755
Rajiv Gandhi International Stadium                     3609
Arun Jaitley Stadium                                   3344
M.Chinnaswamy Stadium                                  3246
Sharjah Cricket Stadium                                2878
Punjab Cricket Association IS Bindra Stadium           2429
Maharashtra Cricket Association Stadium                1467
Holkar Cricket Stadium                                  873
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium     491
Name: venue, dtype: int64

In [114]:
##Filter venue to Wankhede and MAC

#venues=['Wankhede Stadium','MA Chidambaram Stadium']

In [118]:
#cdf=cdf.loc[cdf['venue'].isin(venues)]

In [28]:
##List of all unique teams in the dataset
ipl_teams=cdf.loc[:,['batting_team','bowling_team']].drop_duplicates().melt()['value'].drop_duplicates().tolist()
ipl_teams
##Change Delhi Daredevils to Delhi Capitals
##Change Kings XI Punjab to Punjab Kings

['Royal Challengers Bangalore',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Mumbai Indians',
 'Kings XI Punjab',
 'Rajasthan Royals',
 'Delhi Daredevils',
 'Punjab Kings']

In [29]:
cdf.loc[cdf['batting_team']=='Delhi Daredevils','batting_team']='Delhi Capitals'
cdf.loc[cdf['bowling_team']=='Delhi Daredevils','bowling_team']='Delhi Capitals'
cdf.loc[cdf['batting_team']=='Kings XI Punjab','batting_team']='Punjab Kings'
cdf.loc[cdf['bowling_team']=='Kings XI Punjab','bowling_team']='Punjab Kings'

In [30]:
##Changed teams list

cdf.loc[:,['batting_team','bowling_team']].drop_duplicates().melt()['value'].drop_duplicates().tolist()

['Royal Challengers Bangalore',
 'Chennai Super Kings',
 'Sunrisers Hyderabad',
 'Kolkata Knight Riders',
 'Delhi Capitals',
 'Mumbai Indians',
 'Punjab Kings',
 'Rajasthan Royals']

In [31]:
##Extras column has captured type of extras
##At test time we will not have ball by ball information, do we need to hold a split of extras in that case??
cdf.drop(columns=['wides','noballs','byes','legbyes','penalty'],inplace=True)

### Derive new features

In [32]:
cdf['total_runs']=cdf['runs_off_bat']+cdf['extras']

In [33]:
##Get over number
cdf['over_number']=cdf['ball'].apply(lambda x:int(math.ceil(x)))

In [34]:
def phase_cat(over_number):
    '''Determine phase'''
    if over_number<=6:
        phase="PowerPlay"
    if (over_number>6) & (over_number<=15):
        phase="Middle overs"
    if over_number>15:
        phase="Death overs"
    return phase

In [35]:
cdf['phase']=cdf['over_number'].apply(phase_cat)

In [26]:
#cdf.to_csv('./data/cleaned_cricsheet.csv',index=False)

In [27]:
#cdf.to_excel('./data/cleaned_cricsheet.xlsx',index=False)

### What features can we derive at test time


**Assuming that we can add data to lookup in the submission file**

#### Toss related factors, cannot be found since each file will be sent manually


#### Venue related factors
- Venue specific stats, avg PP score in last 'n' games batting 1st (through a lookup table)
- Venue specific stats, avg PP score in last 'n' games batting 2nd (through a lookup table)

#### Team related factors

- When team is batting 1st, what is the avg PP score
- When team is batting 2nd, what is the avg PP score


#### Batsman related factors

- Since the input is a set of all batters in the list, the order will matter. Assuming openers are 1st two, we can measure size of this set to determine number of wickets that have fallen

- We will not have split of how many balls each batter faced. Hence we need to be careful not to over use the batsman feature

- Lookup batter at venue in 1st innings

- Lookup batter at venue in 2nd innings

- One proxy could be to determine average batting time of batsman in PP by batting position at venue



#### Bowler related factors

- Again we dont have info on the split of balls bowled. 

- Lookup bowler at venue in 1st innings how many overs he has generally bowled * ER
 
- Lookup bowler at venue in 2nd innings how many overs he has generally bowled * ER at venue

#### Batsman-Bowler matchups

- Infer batting hand of Batsman, bowling type of bowlers

#### Handle unknowns

- Bowlers like Chetan Sakariya, Avesh Khan are bowling rarely or for the first time. Model must be robust enough to handle "Unknowns"


### Begin model training pipeline

In [54]:
cdf2=cdf.loc[cdf['phase']=='PowerPlay']

#### Venue related stats

In [88]:
##Venue related sums
venue_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['venue','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()

<ipython-input-88-e63dc6b94306>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  venue_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['venue','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()


In [91]:
venue_df.index=venue_df['start_date']
venue_df.drop(columns=['start_date'],inplace=True)

In [114]:
venue_n=2

In [146]:
vdf_lastn=pd.DataFrame()

for ven in list(set(venue_df['venue'])):
    df=venue_df.loc[venue_df['venue']==ven]
    ##Take the last n innings, multiply by 2 to account for both innings
    df2=df.tail(2*venue_n).groupby('innings')['total_runs','wicket_type'].mean().reset_index()
    df2['venue']=ven
    
    vdf_lastn=pd.concat([vdf_lastn,df2])

<ipython-input-146-3c6b8fb64e3b>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2=df.tail(2*venue_n).groupby('innings')['total_runs','wicket_type'].mean().reset_index()


In [128]:
##To round off values
#vdf_lastn=np.round(vdf_lastn,0)

In [147]:
vdf_lastn

,innings,total_runs,wicket_type,venue
0,1,47.5,1.5,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...
1,2,48.5,0.0,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...
0,1,36.0,0.5,MA Chidambaram Stadium
1,2,50.5,1.0,MA Chidambaram Stadium
0,1,41.5,1.5,Punjab Cricket Association IS Bindra Stadium
1,2,65.0,0.5,Punjab Cricket Association IS Bindra Stadium
0,1,53.0,2.0,Holkar Cricket Stadium
1,2,62.0,1.0,Holkar Cricket Stadium
0,1,36.5,1.5,Narendra Modi Stadium
1,2,42.5,2.5,Narendra Modi Stadium


In [119]:
# vdf=venue_df.groupby(['venue','start_date','match_id','innings']).last(venue_n).reset_index().drop(columns=['match_id'])

# vdf.head()

In [118]:
# list(set(vdf['venue']))

In [79]:
# delhi_df=venue_df.loc[venue_df['venue']=='Arun Jaitley Stadium']

In [120]:
# set(vdf['venue'])

In [121]:
# vdf_lastn=vdf.groupby(['venue','innings'])['total_runs','wicket_type'].mean().reset_index()

# #vdf_lastn=np.round(vdf_lastn,0)
# vdf_lastn.head()

#### Team last few scores

In [168]:
##Team related dimensions

team_n=4
team_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['batting_team','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()

team_df.index=team_df['start_date']
team_df.drop(columns=['start_date'],inplace=True)
# tdf=team_df.groupby(['batting_team','start_date','match_id','innings']).last(team_n).reset_index().drop(columns=['start_date','match_id'])

# tdf.head()

<ipython-input-168-ed7d4fbc9ca5>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  team_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['batting_team','start_date','match_id','innings'])['total_runs','wicket_type'].agg({'total_runs':'sum','wicket_type':'count'}).reset_index()


In [169]:
tdf_lastn=pd.DataFrame()

for team in list(set(team_df['batting_team'])):
    df=team_df.loc[team_df['batting_team']==team]
    ##Take the last n innings, multiply by 2 to account for both innings
    
    df2=pd.concat([df.loc[df['innings']==1].tail(team_n),df.loc[df['innings']==2].tail(team_n)])
    df2=df2.groupby('innings')['total_runs','wicket_type'].mean().reset_index()
    #df2=df.tail(2*team_n).groupby('innings')['total_runs','wicket_type'].mean().reset_index()
    df2['batting_team']=team
    
    tdf_lastn=pd.concat([tdf_lastn,df2])

<ipython-input-169-e148e00a3d54>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2=df2.groupby('innings')['total_runs','wicket_type'].mean().reset_index()
<ipython-input-169-e148e00a3d54>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2=df2.groupby('innings')['total_runs','wicket_type'].mean().reset_index()
<ipython-input-169-e148e00a3d54>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2=df2.groupby('innings')['total_runs','wicket_type'].mean().reset_index()
<ipython-input-169-e148e00a3d54>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2=df2.groupby('innings')['total_runs','wicket_type'].mean().reset_index()
<ipython-input-169-e148e00a3d54>:8: FutureWa

In [170]:
tdf_lastn

,innings,total_runs,wicket_type,batting_team
0,1,50.75,0.50,Sunrisers Hyderabad
1,2,53.25,0.75,Sunrisers Hyderabad
0,1,42.25,2.00,Rajasthan Royals
1,2,45.00,2.25,Rajasthan Royals
0,1,48.25,1.50,Delhi Capitals
1,2,52.25,1.00,Delhi Capitals
0,1,40.00,1.75,Royal Challengers Bangalore
1,2,53.50,1.00,Royal Challengers Bangalore
0,1,38.50,1.50,Punjab Kings
1,2,47.25,1.00,Punjab Kings


In [171]:
# tdf_lastn=tdf.groupby(['batting_team','innings'])['total_runs','wicket_type'].mean().reset_index()

# #tdf_lastn=np.round(tdf_lastn,0)

# tdf_lastn.head()

In [272]:
model_df=cdf.loc[cdf['phase']=='PowerPlay'].groupby(['match_id','innings','venue','batting_team'])['total_runs'].sum().reset_index()

In [273]:
model_df.head()

,match_id,innings,venue,batting_team,total_runs
0,1136561,1,Wankhede Stadium,Mumbai Indians,39
1,1136561,2,Wankhede Stadium,Chennai Super Kings,42
2,1136562,1,Punjab Cricket Association IS Bindra Stadium,Delhi Capitals,45
3,1136562,2,Punjab Cricket Association IS Bindra Stadium,Punjab Kings,73
4,1136563,1,Eden Gardens,Royal Challengers Bangalore,52


In [274]:
model_df=model_df.merge(vdf_lastn,on=['venue','innings'],suffixes=('','_venueavg'))

model_df.tail()

,match_id,innings,venue,batting_team,total_runs,total_runs_venueavg,wicket_type
399,1216521,2,Sharjah Cricket Stadium,Mumbai Indians,52,57.0,0.5
400,1216527,2,Sharjah Cricket Stadium,Rajasthan Royals,69,57.0,0.5
401,1216531,2,Sharjah Cricket Stadium,Punjab Kings,56,57.0,0.5
402,1216538,2,Sharjah Cricket Stadium,Sunrisers Hyderabad,56,57.0,0.5
403,1216540,2,Sharjah Cricket Stadium,Kolkata Knight Riders,43,57.0,0.5


In [275]:
model_df=model_df.merge(tdf_lastn,on=['batting_team','innings'],suffixes=('','_teamavg'))

model_df.tail()

,match_id,innings,venue,batting_team,total_runs,total_runs_venueavg,wicket_type,total_runs_teamavg,wicket_type_teamavg
399,1216528,2,Dubai International Cricket Stadium,Sunrisers Hyderabad,40,46.5,2.5,53.25,0.75
400,1216534,2,Dubai International Cricket Stadium,Sunrisers Hyderabad,48,46.5,2.5,53.25,0.75
401,1216495,2,Sharjah Cricket Stadium,Sunrisers Hyderabad,56,57.0,0.5,53.25,0.75
402,1216502,2,Sharjah Cricket Stadium,Sunrisers Hyderabad,58,57.0,0.5,53.25,0.75
403,1216538,2,Sharjah Cricket Stadium,Sunrisers Hyderabad,56,57.0,0.5,53.25,0.75


In [276]:
##Make innings and Ordinal feature
model_df['innings']=model_df['innings'].astype('str')

### Filter venues?

In [277]:
venue_list=['Wankhede Stadium','MA Chidambaram Stadium','Arun Jaitley Stadium','Narendra Modi Stadium']

In [278]:
#model_df=model_df.loc[model_df['venue'].isin(venue_list)]

In [279]:
# inn_encoder=LabelEncoder()
venue_encoder=LabelEncoder()
team_encoder=LabelEncoder()

In [280]:
model_df['venue_encoded']=venue_encoder.fit_transform(model_df['venue'])

In [281]:
model_df['team_encoded']=team_encoder.fit_transform(model_df['batting_team'])

In [282]:
##Scale values
minmaxscaler=MinMaxScaler()

In [283]:
mdf_x=model_df.loc[:,['innings','total_runs_venueavg','total_runs_teamavg','venue_encoded','team_encoded','wicket_type','wicket_type_teamavg']]

In [323]:
X=minmaxscaler.fit_transform(mdf_x)

In [335]:
mdf_x

,innings,total_runs_venueavg,total_runs_teamavg,venue_encoded,team_encoded,wicket_type,wicket_type_teamavg
0,1,38.0,42.75,13,3,0.5,0.75
1,1,38.0,42.75,13,3,0.5,0.75
2,1,38.0,42.75,13,3,0.5,0.75
3,1,38.0,42.75,13,3,0.5,0.75
4,1,38.0,42.75,13,3,0.5,0.75
...,...,...,...,...,...,...,...
399,2,46.5,53.25,2,7,2.5,0.75
400,2,46.5,53.25,2,7,2.5,0.75
401,2,57.0,53.25,11,7,0.5,0.75
402,2,57.0,53.25,11,7,0.5,0.75


In [324]:
#X=mdf_x

In [325]:
y=np.array(model_df['total_runs'])

In [326]:
from sklearn.model_selection import train_test_split

In [327]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [328]:
model=LinearRegression()

# model=RandomForestRegressor()

In [329]:
# from sklearn.ensemble import RandomForestRegressor

# model=RandomForestRegressor()

In [330]:
model.fit(X_train,y_train)

LinearRegression()

In [331]:
# model.coef_

In [332]:
pred=np.round(model.predict(X_test),0)

In [333]:
from sklearn.metrics import mean_squared_error

In [334]:
mean_squared_error(y_test,pred)

121.1358024691358

In [310]:
pred

array([48., 44., 52., 49., 50., 49., 44., 50., 50., 53., 50., 49., 46.,
       52., 44., 47., 43., 48., 44., 47., 50., 51., 47., 44., 47., 52.,
       46., 44., 46., 45., 49., 46., 45., 46., 50., 53., 48., 50., 46.,
       51., 50., 53., 45., 49., 46., 52., 45., 47., 46., 46., 45., 43.,
       47., 48., 53., 50., 48., 49., 50., 47., 48., 45., 45., 45., 50.,
       48., 51., 46., 43., 52., 49., 50., 49., 45., 45., 45., 48., 45.,
       50., 51., 50.])

In [311]:
y_test

array([36, 42, 68, 51, 59, 22, 49, 43, 58, 46, 34, 36, 67, 35, 36, 34, 69,
       57, 62, 52, 58, 40, 36, 48, 33, 51, 50, 53, 29, 41, 46, 32, 39, 70,
       60, 69, 45, 53, 51, 59, 55, 51, 50, 45, 49, 72, 41, 47, 54, 47, 49,
       52, 36, 51, 51, 31, 46, 41, 37, 61, 36, 42, 63, 59, 43, 28, 48, 53,
       48, 61, 57, 55, 63, 43, 50, 60, 39, 54, 48, 41, 56])

### Save training artefacts for testing pipeline

In [194]:
import joblib

In [195]:
joblib.dump(model,'./submission/model.joblib')

['./submission/model.joblib']

In [196]:
joblib.dump(team_encoder,'./submission/team_encoder.joblib')

['./submission/team_encoder.joblib']

In [197]:
joblib.dump(minmaxscaler, './submission/min_max_scaler.joblib') 

['./submission/min_max_scaler.joblib']

In [198]:
joblib.dump(venue_encoder,'./submission/venue_encoder.joblib')

['./submission/venue_encoder.joblib']

In [199]:
vdf_lastn.to_csv('./submission/vdf.csv',index=False)

In [200]:
tdf_lastn.to_csv('./submission/tdf.csv',index=False)

### Testing pipeline

In [201]:
test_df=pd.read_csv('./submission/inputFile.csv')

# tdf=test_df.loc[:,['venue','innings','batting_team']]

# tdf

In [211]:
##Check wickets lost in innings

wkt_factor=(test_df['batsmen'].apply(lambda x:len(x.split(',')))[0]-2)*7



0

In [67]:
tdf=tdf.merge(vdf_lastn,on=['venue','innings'],suffixes=('','_venueavg'))
tdf=tdf.merge(tdf_lastn,on=['batting_team','innings'],suffixes=('','_teamavg'))

In [68]:
tdf['venue_encoded']=venue_encoder.transform(tdf['venue'])

In [69]:
tdf['team_encoded']=team_encoder.transform(tdf['batting_team'])

In [70]:
tdf.columns=['venue', 'innings', 'batting_team', 'total_runs_venueavg', 'wicket_type',
       'total_runs_teamavg', 'wicket_type_teamavg', 'venue_encoded',
       'team_encoded']

In [71]:
tdf_x=tdf.loc[:,['innings','total_runs_venueavg','total_runs_teamavg','venue_encoded','team_encoded','wicket_type','wicket_type_teamavg']]

In [72]:
tdf_x

,innings,total_runs_venueavg,total_runs_teamavg,venue_encoded,team_encoded,wicket_type,wicket_type_teamavg
0,1,46.785714,45.892857,8,1,1.071429,1.5


In [73]:
Xt=minmaxscaler.transform(tdf_x)

In [74]:
pred=int(np.round(model.predict(Xt),0))

pred

47

In [77]:
y_true=[54]

pred=[pred]

In [78]:
mean_squared_error(y_true,pred)

49.0

### Powerplay specific features for 2018, 2019. Try to use it on 2021

In [ ]:
match_df=cdf.loc[cdf['match_id']==1136565]

In [ ]:
match_df

In [ ]:
pp_score1=int(match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==1)]['total_runs'].sum())
pp_score2=int(match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==2)]['total_runs'].sum())

In [ ]:
pp_batters1=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==1),['striker','non_striker']].melt()['value'].drop_duplicates().tolist()
pp_batters1.sort()
pp_batters2=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==2),['striker','non_striker']].melt()['value'].drop_duplicates().tolist()
pp_batters2.sort()

In [ ]:
pp_batters1, pp_batters2

In [ ]:
pp_bowlers1=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==1),['bowler']].melt()['value'].drop_duplicates().tolist()
pp_bowlers1.sort()


pp_bowlers2=match_df.loc[(match_df['phase']=='PowerPlay')&(match_df['innings']==2),['bowler']].melt()['value'].drop_duplicates().tolist()
pp_bowlers2.sort()

In [ ]:
pp_bowlers1,pp_bowlers2

In [ ]:
match_df